Some manual tests for the qubits system.

Useful to analyze the behavior of the system in different regimes (and assess the performance of the quantum system evolution in TensorFlow).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow.keras as K
import tensorflow_probability as tfp
from tqdm import tqdm, trange
import activelearning as al

In [ ]:
tf_complex = tf.complex128
sigma_x = tf.constant([[0, 1], [1, 0]], dtype=tf_complex)
sigma_y = tf.constant([[0j, -1j], [1j, 0j]], dtype=tf_complex)
sigma_z = tf.constant([[1, 0], [0, -1]], dtype=tf_complex)
tf_kron = tf.experimental.numpy.kron

In [ ]:
dim_x = 1
dim_y = 1
dim_lambda = 5

# System size
n_qubits = dim_lambda

# Hamiltonian parameters
frequencies = [1.0] * n_qubits
coupling = 0.5

# Excited pulse
excited_qubit_idx = 0
alpha_pulse = 2.0

# Evolution parameters
evolution_time = 3
delta_t = 0.01
n_steps = int(evolution_time / delta_t)

# Measured qubit
measurement_idx = 0


tf_n_qubits = tf.convert_to_tensor(n_qubits, tf_complex)
tf_frequencies = tf.convert_to_tensor(frequencies, tf_complex)
tf_coupling = tf.convert_to_tensor(coupling, tf_complex)

tf_alpha_pulse = tf.convert_to_tensor(alpha_pulse, tf_complex)

tf_evolution_time = tf.constant(evolution_time, tf_complex)
tf_delta_t = tf.constant(delta_t, tf_complex)
tf_n_steps = tf.constant(n_steps, tf_complex)

In [ ]:
@tf.function
def tf_kron(tf_A, tf_B):
    tf_shape = tf_A.shape[-1] * tf_B.shape[-1]
    return tf.reshape(
        tf_A[..., :, None, :, None] * tf_B[..., None, :, None, :],
        (-1, tf_shape, tf_shape),
    )


@tf.function
def tf_get_H(tf_frequencies, tf_coupling):
    tf_H_diag = sum(
        tf_kron(
            tf_kron(
                tf.eye(2**i, dtype=tf_complex),
                tf_frequencies[..., i, None, None] * sigma_z,
            ),
            tf.eye(2 ** (n_qubits - i - 1), dtype=tf_complex),
        )
        for i in range(n_qubits)
    )

    tf_H_int = tf_coupling * sum(
        tf_kron(
            tf_kron(
                tf_kron(
                    tf_kron(tf.eye(2**i, dtype=tf_complex), sigma_x),
                    tf.eye(2 ** (j - i - 1), dtype=tf_complex),
                ),
                sigma_x,
            ),
            tf.eye(2 ** (n_qubits - j - 1), dtype=tf_complex),
        )
        for i in range(n_qubits)
        for j in range(i + 1, n_qubits)
    )

    return tf_H_diag + tf_H_int


@tf.function
def tf_ground_state(tf_H):
    tf_eigvals, tf_eigvects = tf.eig(tf_H)
    tf_indices = tf.argmin(tf.math.real(tf_eigvals), axis=-1)
    tf_ground = tf.gather(tf_eigvects, tf_indices, axis=-1, batch_dims=1)
    return tf_ground


@tf.function
def tf_apply_pulse(tf_alpha_pulse, psi):
    U_pulse_ = tf.linalg.expm(-1j * tf_alpha_pulse * sigma_x)
    U_pulse = tf_kron(
        tf_kron(tf.eye(2**excited_qubit_idx, dtype=tf_complex), U_pulse_),
        tf.eye(2 ** (n_qubits - excited_qubit_idx - 1), dtype=tf_complex),
    )[0]
    tf_excited = tf.tensordot(psi, U_pulse, axes=[-1, -1])
    return tf_excited


@tf.function
def schroedinger_rhs(tf_H, tf_psi):
    return -1j * tf.einsum("...ij,...j->...i", tf_H, tf_psi)


@tf.function
def rk_step(tf_H, tf_psi, rhs, tf_dt):
    k1 = rhs(tf_H, tf_psi)
    k2 = rhs(tf_H, tf_psi + 0.5 * tf_dt * k1)
    k3 = rhs(tf_H, tf_psi + 0.5 * tf_dt * k2)
    k4 = rhs(tf_H, tf_psi + tf_dt * k3)
    return (tf_dt / 6.0) * (k1 + 2 * k2 + 2 * k3 + k4)


@tf.function
def tf_evolve_psi(tf_H, tf_psi, tf_dt):
    tf_evolved = tf_psi
    for i in range(n_steps):
        tf_evolved += rk_step(tf_H, tf_evolved, schroedinger_rhs, tf_dt)
    return tf_evolved


@tf.function
def tf_density_mat(tf_psi):
    return tf.einsum("...i,...k->...ik", tf.math.conj(tf_psi), tf_psi)


@tf.function
def tf_measure_probs(tf_rho):
    qubit_reshape = [2**measurement_idx, 2, 2 ** (n_qubits - measurement_idx - 1)]
    tf_rho_qubits = tf.reshape(tf_rho, [-1, *(qubit_reshape * 2)])

    tf_rho_traced = tf.linalg.trace(
        tf.linalg.trace(tf.transpose(tf_rho_qubits, (0, 2, 5, 1, 4, 3, 6)))
    )
    tf_probs = tf.linalg.diag_part(
        tf.einsum("ij,...jk,kl->...il", sigma_z, tf_rho_traced, sigma_z)
    )

    return tf_probs

In [ ]:
evolution_time = 3
delta_t = evolution_time / n_steps

tf_evolution_time = tf.constant(evolution_time, tf_complex)
tf_delta_t = tf.constant(delta_t, tf_complex)
tf_n_steps = tf.constant(n_steps, tf_complex)

In [ ]:
tf_H = tf_get_H(tf_frequencies, tf_coupling)
tf_psi_ground = tf_ground_state(tf_H)
tf_psi_excited = tf_apply_pulse(tf_alpha_pulse, tf_psi_ground)
tf_psi_evolved = tf_evolve_psi(tf_H, tf_psi_excited, tf_delta_t)
tf_rho = tf_density_mat(tf_psi_evolved)
tf_probs = tf.math.real(tf_measure_probs(tf_rho))

In [ ]:
qubit_reshape = [2**measurement_idx, 2, 2 ** (n_qubits - measurement_idx - 1)]
tf_rho_qubits = tf.reshape(tf_rho, [-1, *(qubit_reshape * 2)])

tf_rho_traced = tf.linalg.trace(
    tf.linalg.trace(tf.transpose(tf_rho_qubits, (0, 2, 5, 1, 4, 3, 6)))
)
tf_probs = tf.linalg.diag_part(
    tf.einsum("ij,...jk,kl->...il", sigma_z, tf_rho_traced, sigma_z)
)

In [ ]:
qubit_reshape = [2**measurement_idx, 2, 2 ** (n_qubits - measurement_idx - 1)]
tf_rho_qubits = tf.reshape(tf_rho, [-1, *(qubit_reshape * 2)])

tf_rho_traced = tf.linalg.trace(
    tf.linalg.trace(tf.transpose(tf_rho_qubits, (0, 2, 5, 1, 4, 3, 6)))
)

In [ ]:
evolution_time = 5
delta_t = evolution_time / n_steps

tf_evolution_time = tf.constant(evolution_time, tf_complex)
tf_delta_t = tf.constant(delta_t, tf_complex)
tf_n_steps = tf.constant(n_steps, tf_complex)

with tf.GradientTape() as tape:
    tape.watch([tf_alpha_pulse, tf_frequencies])

    tf_H = tf_get_H(tf_frequencies, tf_coupling)
    tf_psi_ground = tf_ground_state(tf_H)
    tf_psi_excited = tf_apply_pulse(tf_alpha_pulse, tf_psi_ground)
    tf_psi_evolved = tf_evolve_psi(tf_H, tf_psi_excited, tf_delta_t)
    tf_rho = tf_density_mat(tf_psi_evolved)
    tf_probs = tf.math.real(tf_measure_probs(tf_rho))

tape.gradient(tf_probs, tf_alpha_pulse)

In [ ]:
tf_H = tf_get_H(tf_frequencies, tf_coupling)
tf_psi_ground = tf_ground_state(tf_H)
tf_psi_excited = tf_apply_pulse(tf_alpha_pulse, tf_psi_ground)

probs = []
for i in tqdm(np.linspace(0, 5, 100)):
    evolution_time = i
    delta_t = evolution_time / n_steps

    tf_evolution_time = tf.constant(evolution_time, tf_complex)
    tf_delta_t = tf.constant(delta_t, tf_complex)
    tf_n_steps = tf.constant(n_steps, tf_complex)
    tf_psi_evolved = tf_evolve_psi(tf_H, tf_psi_excited, tf_delta_t)
    tf_rho = tf_density_mat(tf_psi_evolved)
    tf_probs = tf.math.real(tf_measure_probs(tf_rho))
    probs.append(tf_probs[:, 0].numpy())

In [ ]:
np.testing.assert_almost_equal(tf.linalg.norm(tf_psi_ground[0]).numpy(), 1.0, decimal=3)
np.testing.assert_almost_equal(
    tf.linalg.norm(tf_psi_excited[0]).numpy(), 1.0, decimal=3
)
np.testing.assert_almost_equal(
    tf.linalg.norm(tf_psi_evolved[0]).numpy(), 1.0, decimal=3
)
np.testing.assert_almost_equal(tf.reduce_sum(tf_probs[0]), 1.0, decimal=3)

In [ ]:
plt.figure()
for i in range(len(probs[0])):
    plt.scatter(np.linspace(0, 5, 100), np.array(probs)[:, i])
plt.show()

Test qubit system parameters

In [ ]:
system = al.systems.get_system_from_name(al.systems.Qubits.__name__)(
    dim_lambda=5, type_lambda="all_ones", x_range=[0, 5], coupling=0.5
)

In [ ]:
system = al.systems.get_system_from_name(al.systems.Qubits.__name__)(
    dim_lambda=5, type_lambda="all_ones", x_range=[0, 5], coupling=0.5
)

In [ ]:
system.plot_response(PLOT_EXTENT=[0, 50])

In [ ]:
system.plot_response(PLOT_EXTENT=[0, 10])

In [ ]:
system.plot_response(PLOT_EXTENT=[0, 5])

In [ ]:
system.plot_response(PLOT_EXTENT=[0, 5])

In [ ]:
system.plot_response(PLOT_EXTENT=[0, 5])

In [ ]:
tf_x_ = tf.cast(0.3, tf_complex)  # represents evolution time
tf_lambda_ = tf.cast(system.tf_real_lambda, tf_complex)  # represents qubit frequencies

tf_H = system.tf_get_H(tf_lambda_, system.tf_coupling)
tf_psi_ground = system.tf_ground_state(tf_H)
tf_psi_excited = system.tf_apply_pulse(system.tf_alpha_pulse, tf_psi_ground)
tf_psi_evolved = system.tf_evolve_psi(tf_H, tf_psi_excited, tf_x_)
tf_rho = system.tf_density_mat(tf_psi_evolved)
tf_probs = tf.math.real(system.tf_measure_probs(tf_rho))

In [ ]:
# tf_H = tf_get_H(tf_frequencies, tf_coupling)
# tf_psi_ground = tf_ground_state(tf_H)
# tf_psi_excited = tf_apply_pulse(tf_alpha_pulse, tf_psi_ground)

probs = []
for i in tqdm(np.linspace(0, 5, 100)):
    tf_x_ = tf.cast(i, tf_complex)  # represents evolution time
    tf_lambda_ = tf.cast(
        system.tf_real_lambda, tf_complex
    )  # represents qubit frequencies

    tf_H = system.tf_get_H(tf_lambda_, system.tf_coupling)
    tf_psi_ground = system.tf_ground_state(tf_H)
    tf_psi_excited = system.tf_apply_pulse(system.tf_alpha_pulse, tf_psi_ground)
    tf_psi_evolved = system.tf_evolve_psi(tf_H, tf_psi_excited, tf_x_)
    tf_rho = system.tf_density_mat(tf_psi_evolved)
    tf_probs = tf.math.real(system.tf_measure_probs(tf_rho))
    probs.append(tf_probs[:, 0].numpy())

In [ ]:
np.array(probs).shape

In [ ]:
plt.figure()
for i in range(len(probs[0])):
    plt.scatter(np.linspace(0, 5, 100), np.array(probs)[:, i])
plt.show()

In [ ]:
np.testing.assert_almost_equal(tf.linalg.norm(tf_psi_ground[0]).numpy(), 1.0, decimal=3)
np.testing.assert_almost_equal(
    tf.linalg.norm(tf_psi_excited[0]).numpy(), 1.0, decimal=3
)
np.testing.assert_almost_equal(
    tf.linalg.norm(tf_psi_evolved[0]).numpy(), 1.0, decimal=3
)
np.testing.assert_almost_equal(tf.reduce_sum(tf_probs[0]), 1.0, decimal=3)

Test likelihood sampling

In [ ]:
learner = al.learners.BayesLearner.from_default(system)

In [ ]:
learner.likelihood.sample(
    10,
    tf_x=tf.convert_to_tensor([3, 4, 5, 5, 6, 7], K.backend.floatx())[:, None, None],
    tf_lambda=learner.system.tf_real_lambda,
)